事前準備
- g++ -std=c++11 -o calc calc.cppを実行し、生成されるcalcファイルを勝率計算時の実行コマンドとして使用する

In [1]:
# -*- coding: utf-8 -*-
import urllib
import requests
import datetime
import subprocess
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
giants_url = "https://baseballdata.jp/1/ctop.html"
swallows_url = "https://baseballdata.jp/2/ctop.html"
dena_url = "https://baseballdata.jp/3/ctop.html"
dragons_url = "https://baseballdata.jp/4/ctop.html"
tigers_url = "https://baseballdata.jp/5/ctop.html"
carp_url = "https://baseballdata.jp/6/ctop.html"

lions_url = "https://baseballdata.jp/7/ctop.html"
fighters_url = "https://baseballdata.jp/8/ctop.html"
lotte_url = "https://baseballdata.jp/9/ctop.html"
orix_url = "https://baseballdata.jp/11/ctop.html"
hawks_url = "https://baseballdata.jp/12/ctop.html"
eagles_url = "https://baseballdata.jp/376/ctop.html"

In [3]:
# dateで受け取った日に開催された各試合の出場成績のリンクをリスト形式で返す
def get_top_links(date):
    params = { 'date': date }
    schedule_page = requests.get('https://baseball.yahoo.co.jp/npb/schedule', params=params)
    soup_schedule = BeautifulSoup(schedule_page.text, 'html.parser')
    game_link_elms = soup_schedule.find_all('a', class_='bb-score__content')
    game_links = list(map(lambda x: x['href'].replace('index', 'top'), game_link_elms))
    return game_links

In [4]:
def data_generate(URL, debug=False):
    html = urllib.request.urlopen(URL)

    # htmlをBeautifulSoupで扱う
    soup = BeautifulSoup(html, "html.parser")
    
    tmp_data= []
    web_data = soup.tbody.findAll("tr")
    for i in range(len(web_data)):
        row_tmp = web_data[i].getText().replace("\r", "").replace(" ", "").split("\n")
        row = [a for a in row_tmp if a != '']
        if row not in tmp_data:
            if "○" in row: #一軍にいる選手
                row.pop(2) #一軍のoマークを削除
                tmp_data.append(row) 
            elif "選手名" in row:
                title = row
            else:
                tmp_data.append(row) #一軍にいないとされている選手
    
    title.remove("調子")
    title.remove("一軍")

    df = pd.DataFrame(tmp_data)    
    df.columns = title
    team = df.iloc[0]["球団"]
    
    if debug:
        display(df.head())
    
    # データの型をobject型からint、またはfloat型に変換する
    for i in ["打点", "本塁打", "安打数", "単打", "2塁打", "3塁打", "得点圏打数", "得点圏安打", 
              "UC本塁打", "試合数", "打席数", "打数", "得点","四球", "死球", "企盗塁", "盗塁", 
              "企犠打","犠打", "犠飛", "代打数", "代打安打", "併殺", "失策", "三振"]:
        df[i] = df[i].astype("int")

    for i in ["打率", "出塁率", "長打率", "最近5試合", "OPS", "得点圏打率", "UC率"]:
        df[i] = df[i].astype("float")

    df["盗塁成功率"] = df["盗塁"] / df["企盗塁"]
    df["犠打成功率"] = df["犠打"] / df["企犠打"]
    df["代打率"] = df["代打安打"] / df["代打数"]
    
    # 欠損値は0で補完
    df["盗塁成功率"].fillna(0, inplace=True)
    df["犠打成功率"].fillna(0, inplace=True)

    #　選手名を修正
    df["選手名"] = df["選手名"].apply(lambda x: x.split(":")[1].split(".")[0].split(team)[0])
    
    # 各選手について 凡退,単打率 二塁打率 三塁打率 本塁打率 四死球率 盗塁成功率 犠打成功率　併殺打率を計算する
    denominator = (df["打席数"] - df["犠打"] - df["犠飛"])
    df["単打率"] = df["単打"] / denominator
    df["二塁打率"] = df["2塁打"] / denominator
    df["三塁打率"] = df["3塁打"] / denominator
    df["本塁打率"] = df["本塁打"] / denominator
    df["四死球率"] = (df["四球"] + df["死球"]) / denominator
    df["併殺打率"] = df["併殺"] / denominator
    df["凡退率"] =  (denominator - df["単打"] - df["2塁打"] - df["3塁打"] - df["本塁打"] - df["四球"] - df["死球"]) / denominator
    df["併殺打率"] = df["併殺"] / denominator

    calc_df = df[["選手名", "凡退率","単打率","二塁打率","三塁打率","本塁打率","四死球率","盗塁成功率","犠打成功率", "併殺打率"]]
    return calc_df, team

In [7]:
# https://stackoverflow.com/questions/31247198/python-pandas-write-content-of-dataframe-into-text-file
def make_df(url, players, opt):
    df, team = data_generate(url)

    players = {i:ind for ind, i in enumerate(players)}
    inv_players = {ind:i for ind, i in enumerate(players)}
    
    df['order'] = df['選手名'].map(players)
    
    #今季一軍初出場選手の調整
    missing_no = list(set(range(9)) - set(df.order.unique()))
    missing_players = [inv_players[i] for i in missing_no] 
    print(missing_players)
    for i in range(len(missing_players)):
        tmp = [1.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.0, 0.000000, 0.000000]
        tmp = pd.DataFrame([missing_players[i]] + tmp + [missing_no[i]]).T
        tmp.columns = df.columns
        df = pd.concat([df, tmp], axis=0)
    
    df = df[~df.order.isnull()].sort_values("order", ascending=True).drop('order', axis=1).reset_index(drop=True)
    
    assert df.shape[0] == 9
    assert np.allclose(df.iloc[:,1:7].sum(axis=1), 1)
    
    df.iloc[:,1:].to_csv(r'data/'+str(opt)+'.txt', header=None, index=None, sep=' ', mode='w')

In [9]:
d_today = datetime.date(2022,5,6)
game_links = get_top_links(d_today)
team_dic = {
    '読売ジャイアンツ':  giants_url ,  
    '東京ヤクルトスワローズ': swallows_url, 
    '横浜DeNAベイスターズ': dena_url, 
    '中日ドラゴンズ': dragons_url, 
    '阪神タイガース': tigers_url, 
    '広島東洋カープ': carp_url,
                           
    '埼玉西武ライオンズ': lions_url, 
    '北海道日本ハムファイターズ': fighters_url, 
    '千葉ロッテマリーンズ': lotte_url, 
    'オリックス・バファローズ': orix_url, 
    '福岡ソフトバンクホークス': hawks_url, 
    '東北楽天ゴールデンイーグルス': eagles_url
}

for id_, game_link in enumerate(game_links):
    html = urllib.request.urlopen(game_link)
    
    soup = BeautifulSoup(html, "html.parser")

    # 対戦チーム名
    home = soup.find('title').text.split(" ")[1].split('vs.')[0]
    visitor = soup.find('title').text.split(" ")[1].split('vs.')[1]
    
    # スタメンを抽出
    data = soup.find_all('td', class_='bb-splitsTable__data bb-splitsTable__data--text')
    data = [i.text.split('\n')[1] for i in data][:20]
    home_member = [i.replace(' ', '') for i in data[1:10]]
    visitor_member = [i.replace(' ', '') for i in data[11:20]]
    
    # データ生成
    print("先攻: " +visitor+ ", 後攻: " +home)
    make_df(team_dic[visitor], visitor_member, "top")
    make_df(team_dic[home], home_member, "bottom")
    
    # 勝率計算
    cmd = "./calc"
    c = subprocess.check_output(cmd).decode()
    print(c)

先攻: 東京ヤクルトスワローズ, 後攻: 読売ジャイアンツ
[]
[]
試合開始の先攻の勝率:0.599923
試合開始の後攻の勝率:0.378197
計算時間 3.218 sec

先攻: 阪神タイガース, 後攻: 中日ドラゴンズ
[]
[]
試合開始の先攻の勝率:0.595326
試合開始の後攻の勝率:0.379723
計算時間 2.920 sec

先攻: 横浜DeNAベイスターズ, 後攻: 広島東洋カープ
[]
[]
試合開始の先攻の勝率:0.482622
試合開始の後攻の勝率:0.497421
計算時間 2.950 sec

先攻: 北海道日本ハムファイターズ, 後攻: 埼玉西武ライオンズ
[]
[]
試合開始の先攻の勝率:0.328779
試合開始の後攻の勝率:0.654379
計算時間 3.132 sec

先攻: 福岡ソフトバンクホークス, 後攻: 千葉ロッテマリーンズ
[]
[]
試合開始の先攻の勝率:0.688945
試合開始の後攻の勝率:0.300379
計算時間 3.195 sec

先攻: 東北楽天ゴールデンイーグルス, 後攻: オリックス・バファローズ
[]
[]
試合開始の先攻の勝率:0.554428
試合開始の後攻の勝率:0.431760
計算時間 2.993 sec

